# Update ontology sources

In [ ]:
!lamin connect laminlabs/bionty-assets

In [ ]:
import bionty as bt
import lamindb as ln
from bionty.base._ontology_url import get_ontology_url
from bionty.core._source import register_source_in_bionty_assets
from lamin_utils import logger

ln.settings.verbosity = "hint"

ln.track("7extigZj6QNG")

All entities that are not listed in the following matrix must be curated manually as they require intervention.
Consult https://bionty-assets-gczz.netlify.app/ingest/ for guidance.

In [ ]:
configs = [
    ("Disease", "mondo", "all"),
    ("CellType", "cl", "all", "2025-04-10"),
    ("Organism", "ncbitaxon", "all"),
    (
        "Tissue",
        "uberon",
        "all",
    ),  # This may take a long time due to parsing, set verbosity to hint to see progress updates
    ("Disease", "doid", "human"),
    ("ExperimentalFactor", "efo", "all"),
    ("Phenotype", "pato", "all"),
    ("Phenotype", "hp", "human"),
    ("Pathway", "go", "all"),
    # ("Pathway", "pw", "all"), Currently leads to a URL error - upstream issue
    ("DevelopmentalStage", "hsapdv", "human"),
    ("DevelopmentalStage", "mmusdv", "mouse"),
    ("Ethnicity", "hancestro", "human"),
    # ("Drug", "dron", "all"), Not a Bionty entity (yet)
]

In [ ]:
for i, config in enumerate(configs, 1):
    entity, source_name, organism, *version = config
    config_id = f"{entity}_{source_name}_{organism}"

    logger.info(f"[{i}/{len(configs)}] Processing {config_id}")

    try:
        *_, version_to_use = get_ontology_url(
            prefix=source_name, version=version[0] if version else None
        )
        new_df = getattr(bt.base, entity)(
            source=source_name, version=version_to_use
        ).df()

        # add some checks here, before I registed the source

        source_rec = getattr(bt, entity).add_source(
            source=source_name, version=version_to_use
        )

        try:
            register_source_in_bionty_assets(
                f"{bt.base.settings.dynamicdir}/df_{organism}__{source_name}__{version_to_use}__{entity}.parquet",
                source=source_rec,
                is_dataframe=True,
            )
            register_source_in_bionty_assets(
                f"{bt.base.settings.dynamicdir}/ontology_{organism}__{source_name}__{version_to_use}__{entity}",
                source=source_rec,
                is_dataframe=False,
            )
        except ValueError as e:
            if "artifact already exists" in str(e):
                logger.warning(f"{config_id} already registered. Skipping...")
                continue
            else:
                raise
        except FileNotFoundError:
            logger.warning(f"{config_id} files not found. Skipping...")
            continue

        logger.info(f"registered {config_id} version {version_to_use}")

    except Exception as e:
        logger.error(
            f"[{i}/{len(configs)}] {config_id} failed: {type(e).__name__}: {str(e)}"
        )
        continue

In [ ]:
ln.finish()